# Collection setup and data load

## Get keys and urls

In [17]:
import os
from dotenv import load_dotenv

load_dotenv()

WEAVIATE_URL = os.getenv("WEAVIATE_URL")
WEAVIATE_KEY = os.getenv("WEAVIATE_KEY")

print(WEAVIATE_URL[:10])
print(WEAVIATE_KEY[:10])

cwzujlbgtk
L1FUbzRPaU


## Connect to Weaviate

You need to pass in your Weaviate Cloud URL and KEY.

In [18]:
import weaviate
from weaviate.classes.init import Auth
# from weaviate.classes.init import AdditionalConfig, Timeout

client = weaviate.connect_to_weaviate_cloud(
    cluster_url=WEAVIATE_URL,
    auth_credentials=Auth.api_key(WEAVIATE_KEY),

    # additional_config=AdditionalConfig(
    #     timeout=Timeout(init=2, query=45, insert=120),  # Values in seconds
    # )
)

client.is_ready()

True

## Create a collection with a vectorizer

* [Weaviate Docs - collection creation and configuration](https://weaviate.io/developers/weaviate/manage-data/collections)
* [Weaviate integrated embedding models](https://weaviate.io/developers/weaviate/model-providers/weaviate/embeddings)

Examples of other embedding models:
* [Cohere](https://weaviate.io/developers/weaviate/model-providers/cohere/embeddings)
* [HuggingFace 🤗](https://weaviate.io/developers/weaviate/model-providers/huggingface/embeddings)
* [Ollama (self-hosted)](https://weaviate.io/developers/weaviate/model-providers/ollama/embeddings)
* [OpenAI](https://weaviate.io/developers/weaviate/model-providers/openai/embeddings)

In [19]:
from weaviate.classes.config import Configure

if client.collections.exists("Jeopardy"):
    client.collections.delete("Jeopardy")

# Create a collection - with Weaviate vectorizer
client.collections.create(
    name="Jeopardy",

    # https://weaviate.io/developers/weaviate/model-providers/weaviate/embeddings
    vector_config=Configure.Vectors.text2vec_weaviate(
        model="Snowflake/snowflake-arctic-embed-l-v2.0",
        dimensions=256 # options 1024 (default) and 256
    ),
)

## Import data
### Sample Data

In [20]:
import json

with open("../jeopardy_tiny.json") as file:
    data_10 = json.load(file)

print(json.dumps(data_10[0:2], indent=2))

[
  {
    "Category": "SCIENCE",
    "Question": "This organ removes excess glucose from the blood & stores it as glycogen",
    "Answer": "Liver"
  },
  {
    "Category": "ANIMALS",
    "Question": "It's the only living mammal in the order Proboseidea",
    "Answer": "Elephant"
  }
]


### Insert Many

> `insert_many` is only used for inserting small batches of data - must complete within the timeout.

[Weaviate Docs - insert many](https://weaviate.io/developers/weaviate/manage-data/import)

In [21]:
# Insert data
jeopardy = client.collections.get("Jeopardy")
jeopardy.data.insert_many(data_10)

BatchObjectReturn(_all_responses=[UUID('d396a838-ca4d-40b2-a693-819e04a1d234'), UUID('d118e37a-e8a7-4824-94e2-9e5531fac643'), UUID('100f6253-5096-4efd-848d-19aa922de886'), UUID('0f6e9865-ecb6-4e0d-b6a3-e49ca383ca23'), UUID('d3a4721c-bbef-4ce1-baa7-ed5d5cc390b8'), UUID('795a525c-1f0d-4500-9582-0e7196b41177'), UUID('88b01f51-1092-4da5-9960-9469ac77f98b'), UUID('ad54f5dd-ef86-41ad-96c8-102636e2575d'), UUID('4a109818-85fd-4b5c-a270-ed424d4ad166'), UUID('5d887076-2d2d-43a7-a264-8f1a4d14fa59')], elapsed_seconds=0.5270140171051025, errors={}, uuids={0: UUID('d396a838-ca4d-40b2-a693-819e04a1d234'), 1: UUID('d118e37a-e8a7-4824-94e2-9e5531fac643'), 2: UUID('100f6253-5096-4efd-848d-19aa922de886'), 3: UUID('0f6e9865-ecb6-4e0d-b6a3-e49ca383ca23'), 4: UUID('d3a4721c-bbef-4ce1-baa7-ed5d5cc390b8'), 5: UUID('795a525c-1f0d-4500-9582-0e7196b41177'), 6: UUID('88b01f51-1092-4da5-9960-9469ac77f98b'), 7: UUID('ad54f5dd-ef86-41ad-96c8-102636e2575d'), 8: UUID('4a109818-85fd-4b5c-a270-ed424d4ad166'), 9: UUID('5

### Data preview

In [22]:
# Show data preview
jeopardy = client.collections.get("Jeopardy")
response = jeopardy.query.fetch_objects(limit=4)

for item in response.objects:
    print(item.uuid, item.properties)

0f6e9865-ecb6-4e0d-b6a3-e49ca383ca23 {'answer': 'Antelope', 'question': 'Weighing around a ton, the eland is the largest species of this animal in Africa', 'category': 'ANIMALS'}
100f6253-5096-4efd-848d-19aa922de886 {'answer': 'the nose or snout', 'question': 'The gavial looks very much like a crocodile except for this bodily feature', 'category': 'ANIMALS'}
4a109818-85fd-4b5c-a270-ed424d4ad166 {'answer': 'the atmosphere', 'question': 'Changes in the tropospheric layer of this are what gives us weather', 'category': 'SCIENCE'}
5d887076-2d2d-43a7-a264-8f1a4d14fa59 {'answer': 'Sound barrier', 'question': 'In 70-degree air, a plane traveling at about 1,130 feet per second breaks it', 'category': 'SCIENCE'}


In [23]:
# Show data preview - with vectors
jeopardy = client.collections.get("Jeopardy")
response = jeopardy.query.fetch_objects(
    limit=4,
    include_vector=True
)

for item in response.objects:
    print(item.properties)
    print(item.vector, '\n')

{'answer': 'Antelope', 'question': 'Weighing around a ton, the eland is the largest species of this animal in Africa', 'category': 'ANIMALS'}
{'default': [0.011283068917691708, 0.028163250535726547, 0.032486945390701294, -0.0235137976706028, 0.004464363679289818, -0.005023334641009569, 0.004216343630105257, -0.08925766497850418, -0.03304961696267128, -0.045783791691064835, -0.03435264900326729, -0.06005790829658508, -0.07918877899646759, 0.02924417331814766, 0.004793823696672916, -0.008536336943507195, 0.019649123772978783, -0.042555827647447586, -0.07735268771648407, -0.043148115277290344, -0.05564536899328232, 0.030858157202601433, -0.06793532520532608, -0.05123283341526985, 0.017043061554431915, -0.06515157967805862, 0.08238713443279266, 0.05132167413830757, 0.06775764375925064, -0.09885270893573761, 0.04664260894060135, 0.020389482378959656, 0.0023321304470300674, 0.03562606871128082, -0.020656011998653412, -0.01012070570141077, -0.0007426724769175053, 0.019338171929121017, -0.0227

### Super quick query example

In [24]:
response = jeopardy.query.near_text(
    query="African animals",
    # query="weather",
    limit=2
)

for item in response.objects:
    print(item.properties)

{'answer': 'Antelope', 'question': 'Weighing around a ton, the eland is the largest species of this animal in Africa', 'category': 'ANIMALS'}
{'answer': 'Elephant', 'question': "It's the only living mammal in the order Proboseidea", 'category': 'ANIMALS'}


## A bit bigger example - 2k objects

### Load data

In [25]:
import json

with open("../wiki-2k.json") as file:
    data_2k = json.load(file)

print(json.dumps(data_2k[0:2], indent=2))

[
  {
    "text": "At this point in the siege, Lee's army had strengthened the Petersburg line. They dug breastworks out of rifle pits. At night, with pick and shovel, they then turned the breastworks into  deep trenches. Pointed stakes turned outwards were designed to break up any frontal attacks. The area between the two lines became a no man's land. The summer that year was hot and dry. Streams and springs were quickly drying up causing a water shortage on both sides. The siege was quickly becoming a stalemate.",
    "title": "Siege of Petersburg",
    "url": "https://simple.wikipedia.org/wiki/Siege%20of%20Petersburg",
    "wiki_id": "20231101.simple_550339_9"
  },
  {
    "text": "1944  Holocaust: Anne Frank and her family are placed on the last transport train from the Westerbork transit camp to Auschwitz.",
    "title": "September 3",
    "url": "https://simple.wikipedia.org/wiki/September%203",
    "wiki_id": "20231101.simple_8532_17"
  }
]


### Create a collection with Named Vectors and SourceProperties

In [26]:
from weaviate.classes.config import Configure, Property, DataType

def create_wiki_collection():
    if client.collections.exists("Wiki"):
        client.collections.delete("Wiki")

    # Create a collection here - with Weaviate vectorizer and define source properties
    client.collections.create(
        name="Wiki",

        vector_config=[
            Configure.Vectors.text2vec_weaviate(
                name="main_vector",
                model="Snowflake/snowflake-arctic-embed-l-v2.0", # default
                source_properties=['title', 'text'] # which properties should be used to generate a vector
            )
        ],

        # Example: how to define property schema (Optional)
        # properties=[  
        #     Property(name="title", data_type=DataType.TEXT),
        #     Property(name="text", data_type=DataType.TEXT),
        #     Property(name="url", data_type=DataType.TEXT),
        #     Property(name="wiki_id", data_type=DataType.TEXT),
        # ],
    )

create_wiki_collection()

### Import data - 2k objects with Batch

Batch speeds up the import process by grouping objects to be added in bigger batch groups.

Batch creates an internal buffer to collect objects to be added.<br>
Each time the buffer count reaches `batch_size`, batch sends the new objects to Weaviate.

Types of batch:
* `dynamic` - let batch calculate the optimal batch_size based on detected latency
* `fixed_size` - provide a fixed batch_size
* `rate_limit` - limit the number of requests (per minute), useful for working with models with a rate limit

### Take 1 – import sample 100

In [27]:
from tqdm import tqdm

sample_100 = data_2k[0:100]

wiki = client.collections.get("Wiki")

with wiki.batch.dynamic() as batch:
    for item in tqdm(sample_100):
        batch.add_object(item)

print(f"Wiki count: {len(wiki)}")

100%|██████████| 100/100 [00:00<00:00, 6829.67it/s]


Wiki count: 100


In [28]:
# check for errors
if(len(wiki.batch.failed_objects)>0):
    print("Import complete with errors")
    for err in wiki.batch.failed_objects:
        print(err)
else:
    print("Import complete with no errors")

Import complete with no errors


### Take 2 – import sample 100 – with UUID

To avoid inserting duplicates, you can generate a UUID based on the whole object or a unique property.

In [29]:
from weaviate.util import generate_uuid5

print(generate_uuid5("This UUID is always the same"))
print(generate_uuid5("This UUID is always the same"))
print(generate_uuid5("This UUID is always the same"))
print("====================================")

print(generate_uuid5("This UUID is different"))
print(generate_uuid5("This UUID is different"))
print("====================================")

obj1 = { "title": "this is an object", "count": 1 }
obj2 = { "title": "this is an object", "count": 2 }
print(generate_uuid5(obj1))
print(generate_uuid5(obj2))


8d3441c0-c1d1-5859-8a5e-efce9e7d3bd8
8d3441c0-c1d1-5859-8a5e-efce9e7d3bd8
8d3441c0-c1d1-5859-8a5e-efce9e7d3bd8
09f975a6-0e62-565a-982e-e6ce148eac86
09f975a6-0e62-565a-982e-e6ce148eac86
c3c3ad32-fa65-5944-a021-415f8fda02af
4d0b77d3-4862-59bc-bf9f-9fe2b9bf89f0


In [30]:
# recreate the collection to start again
create_wiki_collection()

> Rerun the import script multiple times.

> Starting from the second run, the script should finish a lot faster, and the wiki count shouldn't increase.

In [31]:
from tqdm import tqdm
from weaviate.util import generate_uuid5

sample_100 = data_2k[0:100]

wiki = client.collections.get("Wiki")

with wiki.batch.fixed_size(batch_size=20, concurrent_requests=2) as batch:
    for item in tqdm(sample_100):
        id = generate_uuid5(item["wiki_id"])

        batch.add_object(
            item,
            uuid=id
        )

print(f"Wiki count: {len(wiki)}")

100%|██████████| 100/100 [00:00<00:00, 144.07it/s]


Wiki count: 100


### Take 2 - import the rest of the data - but break if multiple errors

In [32]:
from tqdm import tqdm
from weaviate.util import generate_uuid5

wiki = client.collections.get("Wiki")

with wiki.batch.fixed_size(batch_size=2000, concurrent_requests=2) as batch:
    for item in tqdm(data_2k):
        id = generate_uuid5(item["wiki_id"])
        batch.add_object(item, uuid=id)

        # Check number of errors while running
        if(batch.number_errors > 10):
            print("Errors during batch import")
            break

100%|██████████| 2000/2000 [00:00<00:00, 51853.23it/s]


### Check for errors

In [33]:
if(len(wiki.batch.failed_objects)>0):
    print("Import complete with errors")
    for err in wiki.batch.failed_objects:
        print(err)
else:
    print("Import complete with no errors")

Import complete with no errors


## Bonus - iterate through all collection data

The client has a built-in function that allows you to iterate through all collection data.

In [34]:
wiki = client.collections.get("Wiki")

counter = 100

for item in wiki.iterator():
    print(item.properties)

    if (counter == 0): break
    
    counter -= 1

{'text': 'Mission Hills is an urban residential community of the San Fernando Valley, within the city of Los Angeles, California.', 'title': 'Mission Hills, Los Angeles', 'wiki_id': '20231101.simple_512123_0', 'url': 'https://simple.wikipedia.org/wiki/Mission%20Hills%2C%20Los%20Angeles'}
{'text': 'Pavlov researched classical conditioning through the use of dogs and their natural ability to salivate, produce water in their mouths. Thorndike and Watson rejected looking at one\'s own conscious thoughts and feelings ("Introspection"). They wanted to restrict psychology to experimental methods. Skinner\'s research leant mainly on behavior shaping using positive reinforcement (rewards rather than punishments).', 'title': 'Behaviorism', 'wiki_id': '20231101.simple_260848_4', 'url': 'https://simple.wikipedia.org/wiki/Behaviorism'}
{'text': 'On 27 April 1970 he was selected as a cosmonaut and flew as Commander on Soyuz T-5. He retired on 31 October 1992 due to medical reasons after injuries he 

You can also get `vector embeddings`, by using `include_vector`.

In [35]:
counter = 10

for item in wiki.iterator(include_vector=True):
    print(item.properties)
    print(item.vector)

    if (counter == 0): break
    
    counter -= 1

{'text': 'Mission Hills is an urban residential community of the San Fernando Valley, within the city of Los Angeles, California.', 'title': 'Mission Hills, Los Angeles', 'wiki_id': '20231101.simple_512123_0', 'url': 'https://simple.wikipedia.org/wiki/Mission%20Hills%2C%20Los%20Angeles'}
{'main_vector': [-0.04594915360212326, 0.04193487763404846, 0.04189903289079666, -0.0003488972724881023, 0.017983609810471535, -0.021612590178847313, 0.02356596663594246, -0.025286372750997543, -0.022867051884531975, 0.008664748631417751, -0.05046521872282028, 0.009910251013934612, 0.012741751037538052, -0.01751766726374626, 0.010895899496972561, -0.10021360218524933, 0.06365498900413513, -0.06265142560005188, 0.0028359803836792707, 0.018781090155243874, -0.06383419781923294, -0.003908993676304817, 0.009731042198836803, -0.02849421091377735, 0.040644571185112, -0.05121789500117302, 0.038135647773742676, -0.023350916802883148, -0.013861806131899357, -0.07899527251720428, -0.07820675522089005, -0.0289422

## Close the client

In [36]:
client.close()